In [1]:
import os
import sys
import cv2
import json
import shutil
from glob import glob
from tqdm import tqdm
from pathlib import Path

from utils.table2label import table2layout, fuse_gt_info, judge_error
from utils.table_helper import correct_table



def table2label(table_dir, label_dir, error_file_path):
    table_error = {}
    json_files = sorted(glob(os.path.join(table_dir, '*.json')))
    for idx, json_path in tqdm(enumerate(json_files), total=len(json_files)):
        json_dir = os.path.dirname(json_path)
        json_name = os.path.basename(json_path)
        # json_name = "06615.json"
        # json_path = os.path.join(json_dir, json_name)

        json_id = json_name.split('.')[0]
        table = json.load(open(json_path, 'r'))
        
        if not table['is_wireless']:
            continue

        # table['is_wireless'] = True

        # table = correct_table(table)
        try:
            gt_label = table2layout(table)
        except:
            table_error[json_id] = 'table2layout error'
            continue

        ## 有线表格得到的bbox还是cell框，不是text框
        try:
            gt_label = fuse_gt_info(gt_label, table)
        except:
            table_error[json_id] = "fuse_gt_info error" # 仅有1cell的有线表格 滤过
            continue

        valid, msg = judge_error(table, gt_label)
        if not valid:
            print(json_name, msg)
            table_error[json_id] = msg
            continue

        gt_json_path = os.path.join(label_dir, f'{json_id}-gt.json')
        json.dump(gt_label, open(gt_json_path, 'w'), indent=4)
    json.dump(table_error, open(error_file_path, 'w'), indent=4)

    print('table error: {}'.format(len(table_error)))

## STEP.1 gen_gt_labels

In [2]:
'''
输入
    训练集目录: {dataset_root}/train
输出
    训练集标注目录: {dataset_root}/train_gt_json/
    {dataset_root}/train_error.json

使用 fix_gt_table 前 报错的：684
使用 fix_gt_table 后 报错的：665
    + 清除area==0的数据：664
'''
DATASET = "train_jpg480max"
DATASET_ROOT = '/media/ubuntu/Date12/TableStruct/new_data'

TABLE_DIR = os.path.join(DATASET_ROOT, DATASET)
LABEL_DIR = os.path.join(DATASET_ROOT, f'{DATASET}_wireless_gt_json')
TABLE_ERROR_PATH = os.path.join(DATASET_ROOT, f'{DATASET}_wireless_error.json')

if os.path.exists(LABEL_DIR):
    shutil.rmtree(LABEL_DIR)
Path.mkdir(Path(LABEL_DIR), parents=True)

table2label(TABLE_DIR, LABEL_DIR, TABLE_ERROR_PATH)


  0%|          | 36/12104 [00:01<08:01, 25.08it/s]

00029.json line idx:[[6], [12], [113]] not find


  1%|          | 61/12104 [00:02<09:21, 21.45it/s]

00057.json line idx:[[34]] not find


  1%|          | 93/12104 [00:03<05:51, 34.21it/s]

...

06543.json line idx:[[6], [8]] not find


 54%|█████▍    | 6589/12104 [02:16<02:55, 31.46it/s]

06584.json line idx:[[15]] not find


100%|██████████| 12104/12104 [02:34<00:00, 78.27it/s] 

table error: 133
